In [11]:
import math
import numpy as np
from source.subtasks_datasets import *
from source.utils import *
from source.mb_regression import MusicBertRegressor
from source.sound_transforms import *

transform = toLogMelSpectrogram()
train_dataset = EmoMusicDataset(datatype="train", transform=transform, use_cache=True)
test_dataset = EmoMusicDataset(datatype="test", transform=transform, use_cache=True)

n_workers = 0
train_dataloader = just_load(train_dataset, workers=n_workers, batch_size=16)
test_dataloader = just_load(test_dataset, workers=n_workers, batch_size=16)

sound_features = train_dataset.get_features_size(log=True)
target_size = 2
evals = 1

print("Using %d features"%sound_features)

model = MusicBertRegressor(sound_features, target_size).cuda()
#model.BERT.load_pretrained()

== Item shapes ==
song_features:  (3880, 128)
sample_rate:  44100
target:  [3.1, 3.0]
Using 128 features


In [15]:
from sklearn.metrics import r2_score

def evaluate(output, target):
    return r2_score(target, output, multioutput='raw_values')

curve_bert = model.train_model(train_dataloader,
                               test_dataloader,
                               epochs = 50,
                               eval_per_epoch=evals,
                               eval_fn=evaluate)

1899 steps: validation=[0.50027322 0.11784353], training=[0.90692465 0.86313557]                   


In [16]:
%matplotlib widget

plot_curve(model.validation_curve, evals, color="", metric_label="MSE")
plot_curve(model.training_curve, evals, color="", metric_label="MSE")

plt.legend(["Validation Arousal", "Validation Valence", "Train Arousal", "Train Valence"])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Model MSE: 21.93%
Model MSE: 80.14%


In [14]:
c = np.array(model.validation_curve)

np.mean(c[-10:], axis = 0)

array([0.29270644, 0.03394138])